In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import torch
from PIL import Image
import os
import re
from tqdm import tqdm, tqdm_notebook
from torch.utils.data import Dataset

import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
from mlxtend.preprocessing import TransactionEncoder
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import tensorflow as tf
from keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
import pandas as pd
import numpy as np
import unicodedata
from numpy.linalg import norm
import math
import time
import cv2
import pickle

### 장르 - AutoEncoder를 통한 임베딩 벡터 추출
#### 장르를 one-hot encoding -> latent space -> 임베딩 벡터

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/캡스톤/데이터/연극+뮤지컬 (+포스터).xlsx')
df = df.drop(df.columns[0], axis=1)
df.head(3)

,Title,줄거리,장르,Image_path,Image,image_name
0,연극 〈운빨로맨스〉- 대학로,점에 살고 점에 죽는 점보늬의 호랑이띠 남자와 하룻밤 보내기 프로젝트 목숨이 걸린 ...,"로맨스, 코믹, 공감/힐링",/content/drive/MyDrive/캡스톤/데이터/연극/연극_포스터,연극 〈운빨로맨스〉- 대학로.png,/content/drive/MyDrive/캡스톤/데이터/연극/연극_포스터/연그...
1,(리얼타임 코믹연극) 택시안에서 - 서울,쾌활하고 유쾌한 택시 운전사 민수 그리고 하영과 소희 두 남녀의 운명적인 만남!\n...,"코믹, 로맨스, 공감/힐링",/content/drive/MyDrive/캡스톤/데이터/연극/연극_포스터,(리얼타임 코믹연극) 택시안에서 - 서울.png,/content/drive/MyDrive/캡스톤/데이터/연극/연극_포스터/(리어...
2,(코믹연극) 달동네-부산,"정음의 아버지 경민은 행정착오로 인해 월남전 참전 중 전사자 처리가 되고,........","코믹, 감동, 드라마",/content/drive/MyDrive/캡스톤/데이터/연극/연극_포스터,(코믹연극) 달동네-부산.gif,/content/drive/MyDrive/캡스톤/데이터/연극/연극_포스터/(코미...


In [ ]:
def load_pickle(filename):
    with open(filename, 'rb') as f:
        res = pickle.load(f)
    return res
def save_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, x_data):
        self.x_data = x_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        return x

In [ ]:
class AE(nn.Module):
    def __init__(self, input_size):
        super(AE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )

        self.decoder = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, input_size)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
def train(model, train_loader, optimizer, criterion):
    model.train()
    train_loss = 0

    for batch_idx, (feature) in enumerate(train_loader):

      feature = feature
      target = feature
      optimizer.zero_grad()
      encoded, decoded = model(feature)
      loss = criterion(decoded, target)
      loss.backward()
      optimizer.step()

      train_loss += loss.item()

    train_loss /= len(train_loader)
    return train_loss

In [ ]:
def evaluate(model, train_loader):
    model.eval()
    encoded_result = []
    decoded_result = []

    with torch.no_grad():

      for feature in train_loader:
        feature = feature
        encoded, decoded = model(feature)

        encoded_result.append(encoded.cpu().numpy())
        decoded_result.append(decoded.cpu().numpy())

    encoded_result = np.concatenate(encoded_result)
    decoded_result = np.concatenate(decoded_result)
    return encoded_result, decoded_result

In [ ]:
df['장르']

0       로맨스, 코믹, 공감/힐링
1       코믹, 로맨스, 공감/힐링
2          코믹, 감동, 드라마
3      로맨스, 공감/힐링, 판타지
4              감동, 드라마
            ...       
122             감동, 가족
123         관객 참여형, 코믹
124             감동, 가족
125         로맨스, 공감/힐링
126            판타지, 종교
Name: 장르, Length: 127, dtype: object

In [ ]:
df['장르'] = df['장르'].str.replace('/', ', ')

# 각 항목을 리스트로 변환
def convert_to_list(genre_string):
    return genre_string.split(', ')
df['장르'] = df['장르'].apply(convert_to_list)

In [ ]:
df['장르']

0       [로맨스, 코믹, 공감, 힐링]
1       [코믹, 로맨스, 공감, 힐링]
2           [코믹, 감동, 드라마]
3      [로맨스, 공감, 힐링, 판타지]
4               [감동, 드라마]
              ...        
122              [감동, 가족]
123          [관객 참여형, 코믹]
124              [감동, 가족]
125         [로맨스, 공감, 힐링]
126             [판타지, 종교]
Name: 장르, Length: 127, dtype: object

##### TranscationEncoder()
##### sklearn의 OneHotEncoder, LabelEncoder 등과 거의 유사한 Encoder 클래스로 transaction data를 numpy array로 인코딩

In [ ]:
def ae_genre(df):
    df['장르'] = df['장르'].str.replace('/', ', ')
    df['장르'] = df['장르'].apply(convert_to_list)
    te = TransactionEncoder()
    te_ary = te.fit(df['장르'].values).transform(df['장르'].values)
    df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

    # 입력 크기를 맞추기 위해 크기를 35로 조정
    df_encoded = df_encoded.iloc[:, :35]
    df_encoded.iloc[:, :] = df_encoded.values.astype(int)
    genre_matrix = df_encoded.values

    input_size = df_encoded.shape[1]
    model = AE(input_size=input_size)

    # hyperparameters
    BATCH_SIZE = 16
    EPOCHS = 10

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    train_dataset = CustomDataset(genre_matrix)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)

    for epoch in range(1, EPOCHS + 1):
        train_loss = train(model, train_loader, optimizer, criterion)
        print(f"\n[EPOCH: {epoch}], \tTrain Loss: {train_loss:.4f}")

    final_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
    AE_encoded_result, AE_decoded_result = evaluate(model, final_loader)
    save_pickle(AE_encoded_result, '/content/drive/MyDrive/캡스톤/데이터/genres_vecs.pickle')

In [ ]:
ae_genre(df)


[EPOCH: 1], 	Train Loss: 0.0565

[EPOCH: 2], 	Train Loss: 0.0485

[EPOCH: 3], 	Train Loss: 0.0371

[EPOCH: 4], 	Train Loss: 0.0266

[EPOCH: 5], 	Train Loss: 0.0199

[EPOCH: 6], 	Train Loss: 0.0167

[EPOCH: 7], 	Train Loss: 0.0145

[EPOCH: 8], 	Train Loss: 0.0124

[EPOCH: 9], 	Train Loss: 0.0108

[EPOCH: 10], 	Train Loss: 0.0091


In [ ]:
gen_vecs = load_pickle('/content/drive/MyDrive/캡스톤/데이터/genres_vecs.pickle')
gen_vecs

array([[-0.1811912 ,  0.52933705, -0.2526065 , ...,  0.42428982,
        -0.34360644,  0.14504915],
       [-0.1811912 ,  0.52933705, -0.2526065 , ...,  0.42428982,
        -0.34360644,  0.14504915],
       [-0.16639581, -0.10462555,  0.28942317, ..., -0.2565856 ,
        -0.603117  , -0.07809109],
       ...,
       [-0.15018377, -0.24559212,  0.3026621 , ..., -0.3633618 ,
        -0.11072537, -0.14213105],
       [-0.2177245 ,  0.4107043 , -0.11678484, ...,  0.4757371 ,
        -0.31094345,  0.01781753],
       [ 0.08477031, -0.18339545, -0.1202151 , ...,  0.2526248 ,
        -0.23496172, -0.06008792]], dtype=float32)

In [ ]:
gen_numpy = np.squeeze(np.array(gen_vecs))
gen_numpy = pd.DataFrame(gen_numpy)
gen_numpy

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-0.181191,0.529337,-0.252607,0.290807,0.010670,-0.049817,-0.608478,-0.153682,0.188169,-0.383986,...,-0.488176,-0.536137,-0.360789,-0.023883,-0.094975,0.160144,0.009164,0.424290,-0.343606,0.145049
1,-0.181191,0.529337,-0.252607,0.290807,0.010670,-0.049817,-0.608478,-0.153682,0.188169,-0.383986,...,-0.488176,-0.536137,-0.360789,-0.023883,-0.094975,0.160144,0.009164,0.424290,-0.343606,0.145049
2,-0.166396,-0.104626,0.289423,-0.049929,0.345060,0.155126,0.219864,0.238216,0.263331,-0.106263,...,-0.165251,-0.045039,-0.008208,0.103411,-0.243114,0.262168,-0.073584,-0.256586,-0.603117,-0.078091
3,-0.082170,0.218655,-0.287737,0.113580,-0.151919,-0.335267,-0.592100,-0.103464,0.085724,-0.027631,...,-0.161067,-0.451708,-0.321326,0.184836,0.050427,0.397227,0.308948,0.729516,-0.448372,0.000607
4,-0.148182,-0.227868,0.455223,-0.138691,-0.048634,-0.063540,0.220983,0.383860,0.047278,0.106972,...,-0.073543,0.052268,-0.124419,0.169519,-0.097039,0.354695,0.287193,-0.155501,-0.525974,-0.163065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,-0.150184,-0.245592,0.302662,-0.177781,0.110878,-0.292643,0.173735,0.319690,0.271339,0.048180,...,-0.075302,0.249040,-0.277509,0.077261,-0.129987,0.179972,0.232478,-0.363362,-0.110725,-0.142131
123,-0.044464,0.158225,-0.097451,0.131459,0.344813,0.136027,0.019874,-0.192492,0.208756,-0.040043,...,0.020620,0.024481,0.144233,0.038503,-0.020878,-0.163679,-0.294169,-0.143883,-0.129140,-0.012285
124,-0.150184,-0.245592,0.302662,-0.177781,0.110878,-0.292643,0.173735,0.319690,0.271339,0.048180,...,-0.075302,0.249040,-0.277509,0.077261,-0.129987,0.179972,0.232478,-0.363362,-0.110725,-0.142131
125,-0.217725,0.410704,-0.116785,0.180510,-0.296180,-0.287475,-0.558164,-0.004229,-0.050381,-0.188364,...,-0.362353,-0.412825,-0.441157,0.024870,0.066581,0.256557,0.318089,0.475737,-0.310943,0.017818
